# IFU / IWU 

20241010 

1. 전체샷에 대해  unique_id, TEST별로 X_reg_avg 계산.  이걸 M3S
2. Patial shot은 제거하고, Full shot에서만 test no별 avg계산해서 ifu 계산
3. IWU는 모든 SHOT에 대해서 계산. 
4. IFU, IWU 정리.  병합할때 조심해야함.  merge쓰면 정렬순서 깨지고 data도 늘어남;   concat으로 병합할것. 




In [5]:
import pandas as pd
import numpy as np

def compute_is_partial_shot(df, wafer_radius):
    """
    각 샷이 Partial Shot인지 여부를 계산하고 데이터프레임에 추가합니다.
    """
    partial_shot_info = []

    # UNIQUE_ID별로 그룹화하여 STEP_PITCH 값을 사용
    for unique_id, group in df.groupby('UNIQUE_ID'):
        step_pitch_x_value = group['STEP_PITCH_X'].iloc[0]
        step_pitch_y_value = group['STEP_PITCH_Y'].iloc[0]
        
        for _, row in group.iterrows():
            dx, dy = row['DieX'], row['DieY']
            x, y = row['fcp_x'], row['fcp_y']
            
            bottom_left = (x - step_pitch_x_value / 2, y - step_pitch_y_value / 2)
            bottom_right = (x + step_pitch_x_value / 2, y - step_pitch_y_value / 2)
            top_left = (x - step_pitch_x_value / 2, y + step_pitch_y_value / 2)
            top_right = (x + step_pitch_x_value / 2, y + step_pitch_y_value / 2)
            
            corners = [bottom_left, bottom_right, top_left, top_right]
            distances = [np.sqrt(c[0]**2 + c[1]**2) for c in corners]
            
            is_partial = any(dist > wafer_radius for dist in distances)
            
            shot_info = {
                "UNIQUE_ID": unique_id,
                "DieX": dx, 
                "DieY": dy,
                "Is Partial Shot": is_partial,
            }
            
            partial_shot_info.append(shot_info)
    
    # partial_shot_info 리스트를 DataFrame으로 변환
    partial_shot_df = pd.DataFrame(partial_shot_info)
    
    # 기존 DataFrame과 Is Partial Shot 열을 병합
    df_merged = pd.concat([df, partial_shot_df['Is Partial Shot']], axis=1)

    return df_merged

def compute_non_partial_averages(df):
    """
    Partial Shot이 아닌 데이터에 대해 X_reg와 Y_reg의 평균을 계산합니다.
    """
    # Partial Shot이 아닌 데이터만 필터링
    df_non_partial = df[df['Is Partial Shot'] == False]
    
    # UNIQUE_ID와 TEST별로 평균 계산
    df_non_partial_avg = df_non_partial.groupby(['UNIQUE_ID', 'TEST']).agg({
        'X_reg': 'mean',
        'Y_reg': 'mean'
    }).reset_index()
    df_non_partial_avg.rename(columns={'X_reg': 'X_reg_avg', 'Y_reg': 'Y_reg_avg'}, inplace=True)
    return df_non_partial_avg

def compute_ifu(df_non_partial_avg):
    """
    각 UNIQUE_ID별로 IFU_x와 IFU_y를 계산합니다.
    """
    ifu_values = []
    for unique_id, group in df_non_partial_avg.groupby('UNIQUE_ID'):
        mean_x = np.mean(group['X_reg_avg'])
        std_dev_x = np.std(group['X_reg_avg'])
        ifu_x = abs(mean_x) + (3 * std_dev_x)
        
        mean_y = np.mean(group['Y_reg_avg'])
        std_dev_y = np.std(group['Y_reg_avg'])
        ifu_y = abs(mean_y) + (3 * std_dev_y)
        
        ifu_values.append({'UNIQUE_ID': unique_id, 'ifu_x': ifu_x, 'ifu_y': ifu_y})
    df_ifu = pd.DataFrame(ifu_values)
    return df_ifu

def compute_iwu(df, df_non_partial_avg):
    """
    전체 데이터에서 IWU_x와 IWU_y를 계산합니다.
    """
    # 전체 데이터에 non_partial에서 계산한 평균값을 병합
    df_merged = pd.merge(df, df_non_partial_avg, on=['UNIQUE_ID', 'TEST'], how='left')
    
    # X_reg_detrended 및 Y_reg_detrended 계산
    df_merged['X_reg_detrended'] = df_merged['X_reg'] - df_merged['X_reg_avg']
    df_merged['Y_reg_detrended'] = df_merged['Y_reg'] - df_merged['Y_reg_avg']
    
    # UNIQUE_ID별로 IWU 계산
    iwu_values = []
    for unique_id, group in df_merged.groupby('UNIQUE_ID'):
        # IWU_x 계산
        mean_detrended_x = np.mean(group['X_reg_detrended'].dropna())
        std_dev_detrended_x = np.std(group['X_reg_detrended'].dropna())
        iwu_x = abs(mean_detrended_x) + (3 * std_dev_detrended_x)
        
        # IWU_y 계산
        mean_detrended_y = np.mean(group['Y_reg_detrended'].dropna())
        std_dev_detrended_y = np.std(group['Y_reg_detrended'].dropna())
        iwu_y = abs(mean_detrended_y) + (3 * std_dev_detrended_y)
        
        iwu_values.append({'UNIQUE_ID': unique_id, 'iwu_x': iwu_x, 'iwu_y': iwu_y})
    df_iwu = pd.DataFrame(iwu_values)
    return df_iwu

def main():
    # 데이터 불러오기
    file_path = 'RawData-1.csv'
    df = pd.read_csv(file_path)
    
    # Wafer 반경 설정
    wafer_radius = 150000
    
    # Partial Shot 여부 계산
    df_with_partial = compute_is_partial_shot(df, wafer_radius)

    df_with_partial.to_csv('df_with_patial.csv', index=False)
    
    # Partial Shot이 아닌 데이터의 평균 계산
    df_non_partial_avg = compute_non_partial_averages(df_with_partial)
    df_non_partial_avg.to_csv('non_partial_avg.csv', index=False)
    
    # IFU 계산
    df_ifu = compute_ifu(df_non_partial_avg)
    
    # IWU 계산
    df_iwu = compute_iwu(df_with_partial, df_non_partial_avg)
    
    # 결과 출력
    print("IFU 결과:")
    print(df_ifu)
    
    print("\nIWU 결과:")
    print(df_iwu)
    
    # 결과 저장 (필요에 따라 주석 해제)
    # df_ifu.to_csv('ifu_results.csv', index=False)
    # df_iwu.to_csv('iwu_results.csv', index=False)
    
if __name__ == "__main__":
    main()



IFU 결과:
                                           UNIQUE_ID     ifu_x     ifu_y
0  VH075030_PTVB827_3GJPBVH.VH075P_PTXV821_2024-0...  0.001692  0.002208
1  WC046030_PTXV831_54BPPWC.WC046P_DBF_PTVP841_20...  0.001233  0.001577
2  WF075030_PTVP841_5G4PPWF.WF075P_-_2024-07-16 2...  0.001616  0.001756

IWU 결과:
                                           UNIQUE_ID     iwu_x     iwu_y
0  VH075030_PTVB827_3GJPBVH.VH075P_PTXV821_2024-0...  0.002959  0.003143
1  WC046030_PTXV831_54BPPWC.WC046P_DBF_PTVP841_20...  0.001937  0.002036
2  WF075030_PTVP841_5G4PPWF.WF075P_-_2024-07-16 2...  0.004032  0.003747
